# Graph

In this tutorial we introduce the notion of `MultiModalGraph`, how to construct it, manipulate it.

`MultiModalGraph` is made of two component, the `flow_graph` and the `mobility_graph`. They both are an `OrientedGraph`, but one is geometric and the other topologic. 
The `flow_graph` is a geometric representation of the roads. The `mobility_graph` represent each mobility service define on the `flow_graph`.

## Create a simple flow graph

### Instantiation

In [ ]:
import mnms as ms

mmgraph  = ms.MultiModalGraph()
flow_graph = mmgraph.flow_graph

### Adding nodes

In [ ]:
flow_graph.add_node('IN', [-0.5, 0])
flow_graph.add_node('0', [0, 0])
flow_graph.add_node('1', [1, 0])
flow_graph.add_node('2', [1, 1])
flow_graph.add_node('3', [0, 1])
flow_graph.add_node('OUT', [1.5, 1])

flow_graph.nodes

### Adding links

In [ ]:
flow_graph.add_link('IN_0', 'IN', '0')
flow_graph.add_link('0_1', '0', '1')
flow_graph.add_link('1_2', '1', '2')
flow_graph.add_link('0_3', '0', '3')
flow_graph.add_link('3_2', '3', '2')
flow_graph.add_link('2_OUT', '2', 'OUT')

flow_graph.links

### Rendering

In [ ]:
from mnms.tools.render import draw_flow_graph
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16, 9))
draw_flow_graph(ax, flow_graph, nodesize=5, linkwidth=2)

## Add mobility services

We add the nodes and links for the car and bus mobility service, each `TopoNode` in a mobility service can refer to a `GeoNode` of the `flow_graph`.
Each link define a dictionnary of cost, in this example we use the travel time of the links.

### Car mobility service

In [ ]:
SPEED_CAR = 10

car = ms.BaseMobilityService('car', SPEED_CAR)
car.add_node('CIN', 'IN')
car.add_node('C0', '0')
car.add_node('C1', '1')
car.add_node('C2', '2')
car.add_node('COUT', 'OUT')

car.add_link('CIN_C0', 'CIN', 'C0', {'time':0.5/SPEED_CAR}, reference_links=['IN_0'])
car.add_link('C0_C1', 'C0', 'C1', {'time':1/SPEED_CAR}, reference_links=['0_1'])
car.add_link('C1_C2', 'C1', 'C2', {'time':1/SPEED_CAR}, reference_links=['1_2'])
car.add_link('C2_COUT', 'C2', 'COUT', {'time':0.5/SPEED_CAR}, reference_links=['2_OUT'])

### Bus mobility service

In [ ]:
SPEED_BUS = 5

bus = ms.BaseMobilityService('bus', SPEED_BUS)
bus.add_node('BIN', 'IN')
bus.add_node('B0', '0')
bus.add_node('B3', '3')
bus.add_node('B2', '2')
bus.add_node('BOUT', 'OUT')

bus.add_link('BIN_B0', 'BIN', 'B0', {'time':0.5/SPEED_BUS}, reference_links=['IN_0'])
bus.add_link('B0_B3', 'B0', 'B3', {'time':1/SPEED_BUS}, reference_links=['0_3'])
bus.add_link('B3_B2', 'B3', 'B2', {'time':1/SPEED_BUS}, reference_links=['3_2'])
bus.add_link('B2_COUT', 'B2', 'BOUT', {'time':0.5/SPEED_BUS}, reference_links=['2_OUT'])

### Add the mobility services to the MultiModalGraph

Then we add the two mobility services to the `MultiModalGraph`. And check that there is no duplicate in the nodes and links.

In [ ]:
mmgraph.add_mobility_service(car)
mmgraph.add_mobility_service(bus)
mmgraph.mobility_graph.check()

### Rendering the mobility services

In [ ]:
from mnms.tools.render import draw_multimodal_graph

fig, ax = plt.subplots(figsize=(16, 9))
draw_multimodal_graph(ax, mmgraph, nodesize=5, linkwidth=2)

## Compute shortest path

To compute a shortest path, we define a `User` with an origin/destination and a departure time. The shortest path algorithm use the dictionary of cost define on links. We use the `time` cost previously define.

In [ ]:
from mnms.graph.algorithms import compute_shortest_path
from mnms.demand.user import User
from mnms.tools.time import Time

user = User('U0', 'IN', 'OUT', Time('07:00:00'))
cost = compute_shortest_path(mmgraph, user, cost='time')
print(f"Path cost: {cost}, path:{user.path}") 